In [2]:
import sys
if "/Users/raymond/code/FinalProject563" not in sys.path:
    sys.path.append("/Users/raymond/code/FinalProject563")
import pandas
import numpy as np
import torch
import matplotlib.pyplot as plt
import random

import connect_four
import mcts
import connect_net
import connect_plot
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
nnet = connect_net.ConnectNet()
# nnet = connect_net.load_model("model_save_04.model")
optimizer = torch.optim.Adam(nnet.parameters(), lr=0.001)

In [48]:
import cProfile
cProfile.run('mcts.policy_iteration(nnet=nnet, num_iterations=1, num_episodes=8, num_mcts_sims=256)', sort="tottime")


Iteration: 0::0,1,2,3,4,5,6,7,         7422698 function calls (6892608 primitive calls) in 31.293 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
223770/47360   10.561    0.000   30.927    0.001 mcts.py:55(search)
    70736    7.009    0.000    7.009    0.000 {built-in method conv2d}
   220674    1.190    0.000    1.190    0.000 {built-in method rsub}
    36385    1.122    0.000    1.625    0.000 connect_four.py:31(game_ended)
   212980    0.791    0.000    0.791    0.000 {method 'sum' of 'torch._C._TensorBase' objects}
    44264    0.748    0.000    1.252    0.000 connect_four.py:44(next_state)
389048/35368    0.640    0.000   11.583    0.000 module.py:1104(_call_impl)
   176410    0.613    0.000    0.613    0.000 {built-in method argmax}
   176595    0.610    0.000    3.028    0.000 mcts.py:37(next_state_rep)
   176410    0.607    0.000    0.607    0.000 {method 'abs' of 'torch._C._TensorBase' objects}
    70736    0.607

In [38]:
examples = mcts.policy_iteration(nnet=nnet, num_iterations=1, num_episodes=1, num_mcts_sims=128)


Iteration: 0::0,

In [4]:
test_state = np.zeros([6,7], dtype=np.int32)
test_state[5,3] = 1
test_state[5,2] = -1
test_state[4,3] = 1
test_state[3,3] = 1
test_state[2,3] = 1
test_state[0,0] = 1
test_state[1,0] = -1
test_state[2,0] = 1
test_state[3,0] = -1
test_state[4,0] = 1
test_state[5,0] = -1
test_state_tensor = torch.Tensor(test_state).reshape(6, 7)

test = {}
test["12948731587817346817264812764817623"] = 10

In [11]:
N = torch.Tensor([1, 2, 3, 4, 5, 6, 7])
Q = torch.Tensor([1, 2, 3, 4, 5, 6, 7])
P = torch.Tensor([1, 2, 3, 4, 5, 6, 7])
c = 1

In [24]:
def old():
    max_u, best_a = -torch.inf, -1
    # for a in self.nc.actions(state=state, rep=rep):
    for a in connect_four.get_valid_actions(state=test_state_tensor):
        visits = 1 + N[a]
        u_explore = (N.sum() + 1).sqrt() / visits
        u = Q[a] / visits + c * P[a] * u_explore
        if u > max_u:
            max_u = u
            best_a = a
        print(a)
    a = best_a
    return a
old()

1
2
3
4
5
6


6

In [30]:
1-test_state_tensor[0,:]

visits = 1 + N
u_explore = (N.sum() + 1).sqrt() / visits
u = 1 + Q / visits + c * P * u_explore
u = u * (1 - test_state_tensor[0, :])
int(torch.argmax(u))

6

In [31]:
test_state_tensor[0, :]

tensor([1., 0., 0., 0., 0., 0., 0.])

In [27]:
u*(1-test_state_tensor[0,:])

tensor([0.0000, 5.2568, 5.7889, 6.1081, 6.3210, 6.4730, 6.5870])